In [1]:
using Statistics, CSV, DataFrames

In [4]:
# include("data.jl")

data (generic function with 1 method)

In [2]:
include("data.jl")
include("solver.jl")

sc_weight_fw_covariates (generic function with 1 method)

In [3]:
# function contract3(X::Array{Float64,3}, v::Union{Vector,Nothing}=nothing)::Matrix{Float64}
#     if !isnothing(v) && size(X, 3) != length(v)
#       throw(ArgumentError("The length of `v` must match the size of the third dimension of `X`"))
#     end
#     out = zeros(Float64, size(X, 1), size(X, 2))
#     if isnothing(v)
#       return out
#     end
#     for ii in eachindex(v)
#       out .+= v[ii] * X[:, :, ii]
#     end
#     return out
#   end

In [11]:
# function fw_step(A::Matrix, x::Vector{Float64}; b::Vector{Float64}, eta::Number, alpha::Union{Nothing,Float64}=nothing)::Vector{Float64}
#     Ax = A * x
#     half_grad = transpose(Ax .- b) * A + (eta * x)'
#     i = findmin(half_grad)[2][2]
#     if !isnothing(alpha)
#       x *= (1 - alpha)
#       x[i] += alpha
#       return x
#     else
#       d_x = -x
#       d_x[i] = 1 - x[i]
#       if all(d_x .== 0)
#         return x
#       end
#       d_err = A[:, i] - Ax
#       step_upper = -half_grad * d_x
#       step_bot = sum(d_err .^ 2) + eta * sum(d_x .^ 2)
#       step = step_upper[1] / step_bot
#       constrained_step = min(1, max(0, step))
#       return x + constrained_step * d_x
#     endS
#   end

fw_step (generic function with 1 method)

In [32]:
# df = CSV.read("C:/Users/User/Documents/GitHub/synthdid/data/CPS.csv", DataFrame);

In [5]:
# df1 = data("california_prop99");

In [3]:
df1 = data("california_prop99");
state = []
year = []
perca =[]
treat = []
for i in df1[:,"State"]
    state = vcat(state, i)
end

a = groupby(df1, :Year)
a_1 = DataFrame(permutedims(a[1])[3,:])
for i in 2:length(a)
    append!(a_1,DataFrame(permutedims(a[i])[3,:]))
end
rename!(a_1, Symbol.(state[1:39]))
df = a_1;
Y = df;

In [43]:
X = cat(zeros(size(Y)), dims = ndims(Y)+1);
zeta_lambda = 0; zeta_omega = 0;
lambda_intercept = true; omega_intercept = true;
min_decrease = 1e-3; max_iter = 1000;
lambda = nothing; omega = nothing; beta = nothing; update_lambda = true; update_omega = true

T0 = size(Y)[2] - 1
N0 = size(Y)[1] - 1
if ndims(X) == 2
    cat(X; dims = ndims(X)+1)
end
if isnothing(lambda)
    lambda = repeat([1/T0], T0)
end
if isnothing(omega)
    omega = repeat([1/N0], N0)
end;
if isnothing(beta)
    beta = repeat([0.0], size(X)[3]-1)
    if isempty(beta)
        beta = nothing
    end
end

Y_lambda = zeros(N0, T0+1);
if lambda_intercept
    for i in 1:size(Y,2)
        Y_lambda[:,i] = Y[1:N0,i] .- mean(Y[1:N0,i])
    end
else
    Y_lambda = Y[1:N0,:]
end
# if update_lambda
#     lambda = fw_step(Y_lambda[:, 1:T0], lambda, b = Y_lambda[:,T0+1], eta = N0 * real(zeta_lambda^2))
# end
# ;

In [44]:
fw_step(Y_lambda[:, 1:T0], lambda, b = Y_lambda[:,T0+1], eta = N0 * real(zeta_lambda^2))

38-element Vector{Float64}:
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 ⋮
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667
 0.01787889140103667

---

In [8]:
fw_step(Y_lambda[:, 1:T0], lambda, b = Y_lambda[:,T0+1], eta = N0 * real(zeta_lambda^2), alpha = 0.8)

38-element Vector{Float64}:
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 ⋮
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684
 0.00526315789473684

In [46]:
A = reshape(1:9.0, 3, 3)
x = [0.5, 0.5, 0.5]
b = [1.0, 2.0, 3.0]
eta = 0.1
alpha = nothing
alpha
isnothing(alpha)
# fw_step1(A, x, b, eta)
A = Matrix(A)
fw_step(A, x, b=b, eta=eta, alpha=nothing)

3-element Vector{Float64}:
 0.9997262523952916
 0.00027374760470844084
 0.00027374760470844084

In [20]:
sc_weight_fw()

In [36]:
b = sc_weight_fw(Matrix(Y), 4, lambda = [1:38.00;])

Dict{String, Vector{Float64}} with 2 entries:
  "lambda" => [6.18356e-5, 0.000123671, 0.000185507, 0.0891445, 0.000309178, 0.…
  "vals"   => [239.321, 126.689, 60.136, 23.5928, 20.5949, 19.4619, 17.1198, 16…

In [40]:
b["vals"]

1000-element Vector{Float64}:
 239.32104641750843
 126.68924154157388
  60.13599965491975
  23.592787726045483
  20.594896051336338
  19.461906407280367
  17.11978216481548
  16.062185438427104
  15.510099257866807
  15.41567509791411
   ⋮
  13.245754928499363
  13.245737928245381
  13.245726798616339
  13.245709653099633
  13.245669683697876
  13.245648535919326
  13.245602259937872
  13.245587930210181
  13.245575343395661

In [26]:
size(Y)

(31, 39)

In [30]:
[1:39.00;]

39-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
  ⋮
 31.0
 32.0
 33.0
 34.0
 35.0
 36.0
 37.0
 38.0
 39.0

In [19]:
mutable struct sc_weight_fw_covariates1
    lambda
    omega
    beta
    vals
end

mutable struct update_weights1
    val
    lambda
    omega
    err_lambda
    err_omega
end

function sc_weight_fw_covariates(Y::Matrix; X = cat(zeros(size(Y)), dims = ndims(Y)+1),
                                    zeta_lambda = 0, zeta_omega = 0,
                                    lambda_intercept = true, omega_intercept = true,
                                    min_decrease = 1e-3, max_iter = 1000,
                                    lambda = nothing, omega = nothing, beta = nothing, update_lambda = true, update_omega = true)
    
    if length(size(Y)) == 2 && length(size(X)) == 3 && all(size(Y) == size(X)[1:2]) && all(Matrix((isfinite.(Y)))) && all(isfinite.(X))
        "continue"
    else
    error("the following condition is not met: length(size(Y)) != 2 || length(size(X)) != 3 || all(size(Y) != size(X)[1:2]) || !all(Matrix((isfinite.(Y)))) || !all(isfinite.(X))")
    end
    
    T0 = size(Y)[2] - 1
    N0 = size(Y)[1] - 1
    if ndims(X) == 2
        cat(X; dims = ndims(X)+1)
    end
    if isnothing(lambda)
        lambda = repeat([1/T0], T0)
    end
    if isnothing(omega)
        omega = repeat([1/N0], N0)
    end;
    if isnothing(beta)
        beta = repeat([0.0], size(X)[3]-1)
        if isempty(beta)
            beta = nothing
        end
    end
    
    function update_weights(Y, lambda, omega)

        Y_lambda = zeros(N0, T0+1);
        if lambda_intercept
            for i in 1:size(Y,2)
                Y_lambda[:,i] = Y[1:N0,i] .- mean(Y[1:N0,i])
            end
        else
            Y_lambda = Y[1:N0,:]
        end
        if update_lambda
            lambda = fw_step(Y_lambda[:, 1:T0], lambda, b = Y_lambda[:,T0+1], eta = N0 * real(zeta_lambda^2))
        end
        err_lambda = Y_lambda * vcat(lambda, -1);

        Y_omega = zeros(size(Matrix(Y[:, 1:T0])', 1), size(Matrix(Y[:, 1:T0])',2));
        if omega_intercept
            for i in 1:size(Matrix(Y[:, 1:T0])', 2)
                Y_omega[:,i] = Matrix(Y[:, 1:T0])'[1:T0, i] .- mean(Matrix(Y[:, 1:T0])'[1:T0, i])
            end
        else
            Y_omega = Matrix(Y[:, 1:T0])'
        end
        if update_omega
            omega = fw_step(Y_omega[:, 1:N0], omega, b = Y_omega[:,N0+1], eta = N0 * real(zeta_omega^2))
        end
        err_omega = Y_omega * vcat(omega, -1)
        val = real(zeta_omega.^2) * sum(omega.^2) + real(zeta_lambda.^2) * sum(lambda.^2) + sum(err_omega.^2) / T0 .+ sum(err_lambda.^2) ./ N0
        # return Dict("val" => val, "lambda" => lambda, "omega" => omega, "err_lambda" => err_lambda, "err_omega" => err_omega)
        res1 = update_weights1(val, lambda, omega, err_lambda, err_omega);
        return res1;
        
    end

    vals = repeat([0.0], max_iter);
    t = 0
    Y_beta = Y .- contract3(X, beta);
    weights = update_weights(Y_beta, lambda, omega);

    while t < max_iter && (t < 2 || vals[t - 1] - vals[t] > min_decrease^2)
        t = t + 1
        if size(X)[3]-1 == 0
            grad_beta = 0
        else
            grad_beta = print("error in  while t < max_iter && (t < 2 || vals[t - 1] - vals[t] > min_decrease^2)")
        end
        
        alpha = 1 / t
    
        if isnothing(beta)
            beta = 0
        end
        
        beta = beta .- alpha * grad_beta
        beta = nothing
        Y_beta = Y .- contract3(X, beta)
        weights = update_weights(Y_beta, weights.lambda, weights.omega)
        vals[t] = weights.val
    end
    res2 = sc_weight_fw_covariates1(weights.lambda, weights.omega, beta, vals)
    
    return res2
end 

sc_weight_fw_covariates (generic function with 13 methods)

In [12]:
a = sc_weight_fw_covariates(Matrix(Y));